# Ficheros y persistencia de datos
En sesiones anteriores, hemos explorado diferentes aspectos de Python acompañados de ejemplos. Estos tienen un propósito didáctico, por ello se han ejecutado en un simple shell de Python o en forma de módulo de Python. Se ejecutaban, tal vez imprimían algo en la consola, y luego terminaban, sin dejar rastro de su breve existencia.

Las aplicaciones del mundo real, sin embargo, son muy diferentes. Naturalmente, siguen ejecutándose en memoria, pero interactúan con redes, discos y bases de datos. También intercambian información con otras aplicaciones y dispositivos, utilizando formatos adecuados a la situación.

En esta sesión, vamos a empezar a acercarnos al mundo real explorando lo siguiente:
- [Archivos y directorios]()
- [Compresión]()
- [Redes y flujos]()
- [El formato de intercambio de datos JSON]()
- [Persistencia de datos con pickle y shelve, de la biblioteca estándar]()
- [Persistencia de datos con SQLAlchemy]()

Como de costumbre, se intenta equilibrar amplitud y profundidad para que, al final se pueda tener un sólido conocimiento de los fundamentos y poder saber cómo buscar más información en la red.

# Trabajando con archivos y directorios
Cuando se trata de archivos y directorios, Python ofrece un montón de herramientas útiles. En particular, en los siguientes ejemplos, aprovecharemos los módulos `os`, `pathlib` y `shutil`. Como vamos a leer y escribir en el disco, vamos a utilizar un archivo, fear.txt, que contiene la sinopsis del libro "LAS INTERMITENCIAS DE LA MUERTE" escrito por José Saramago como conejillo de indias para algunos de nuestros ejemplos.

## Abrir archivos
Abrir un archivo en Python es muy sencillo e intuitivo. De hecho, sólo tenemos que utilizar la función `open()`. Veamos un ejemplo rápido:

In [2]:
fh = open('fear.txt', 'rt', encoding='utf-8') # r: read, t: text
for line in fh.readlines():
    print(line.strip())                       # removemos los espacios en blanco e imprimimos
fh.close()

Sinopsis de LAS INTERMITENCIAS DE LA MUERTE
Autor: JOSÉ SARAMAGO
Una brillante sátira que juega con el miedo más profundo del ser humano, el de perder la vida. En un país cuyo nombre no será mencionado se produce algo nunca visto desde el principio del mundo: la muerte decide suspender su trabajo letal, la gente deja de morir. La eurofia colectiva se desata, pero muy pronto dará paso a la desesperación y al caos. Sobran los motivos. Si es cierto que las personas ya no mueren, eso no significa que el tiempo haya parado. el destino de los humanos será una vejez eterna. Se buscarán maneras de forzar a la muerte a matar aunque no lo quiera, se corromperán las conciencias en los "acuerdos de caballeros" explícitos o tácitos entre el poder político, las mafias y las familias, los ancianos serán detestados ppor haberse convertido en estorbos irremovibles. Hasta el día en que la muerte decide volver...


El código anterior es muy sencillo. Llamamos a `open()`, pasándole el nombre del fichero, y diciéndole a `open()` que queremos leerlo en modo texto (además, le pasamos un parámetro `encoding='utf-8'` para el tema de las tildes en el archivo). No hay información de ruta antes del nombre del archivo; por lo tanto, open() asumirá que el archivo está en la misma carpeta en la que se ejecuta el script. Esto significa que si ejecutamos este script desde fuera de la carpeta donde estamos trabajando, el archivo fear.txt no será encontrado.

Una vez que el archivo ha sido abierto, obtenemos de vuelta un objeto archivo, `fh`, que podemos usar para trabajar en el contenido del archivo. En este caso, utilizamos el método `readlines()` para recorrer todas las líneas del fichero e imprimirlas. Llamamos a `strip()` en cada línea para deshacernos de cualquier espacio extra alrededor del contenido, incluyendo el carácter de terminación de línea al final, ya que `print` añadirá uno por nosotros. Esta es una solución rápida y sucia que funciona en este ejemplo, pero si el contenido del archivo contiene espacios significativos que deben ser preservados, tendrá que ser un poco más cuidadoso en la forma de desinfectar los datos. Al final del script, cerramos el flujo.

Cerrar un archivo es muy importante, ya que no queremos arriesgarnos a no liberar el handle (es un identificador o referencia que el sistema proporciona para acceder a un recurso, como un archivo, una ventana, o una conexión a la base de datos, este permite interactuar con ese recurso sin exponer los detalles internos de su implementación) que tenemos sobre él. Cuando eso ocurre, podemos encontrarnos con problemas como fugas de memoria, o la molesta ventana emergente «no puede eliminar este archivo» que nos informa de que algún software todavía lo está utilizando. Por lo tanto, necesitamos aplicar algunas precauciones, y envolver la lógica anterior en un bloque `try/finally`. Esto significa que, sea cual sea el error que se produzca cuando intentemos abrir y leer el archivo, podemos estar seguros de que se llamará a `close()`:

In [5]:
fh = open('fear.txt', 'rt', encoding='utf-8')
try:
    for line in fh.readlines():
        print(line.strip())
finally:
    fh.close()

Sinopsis de LAS INTERMITENCIAS DE LA MUERTE
Autor: JOSÉ SARAMAGO
Una brillante sátira que juega con el miedo más profundo del ser humano, el de perder la vida. En un país cuyo nombre no será mencionado se produce algo nunca visto desde el principio del mundo: la muerte decide suspender su trabajo letal, la gente deja de morir. La eurofia colectiva se desata, pero muy pronto dará paso a la desesperación y al caos. Sobran los motivos. Si es cierto que las personas ya no mueren, eso no significa que el tiempo haya parado. el destino de los humanos será una vejez eterna. Se buscarán maneras de forzar a la muerte a matar aunque no lo quiera, se corromperán las conciencias en los "acuerdos de caballeros" explícitos o tácitos entre el poder político, las mafias y las familias, los ancianos serán detestados ppor haberse convertido en estorbos irremovibles. Hasta el día en que la muerte decide volver...


La lógica es exactamente la misma al primer ejercicio, pero este le brinda mayor seguridad a la hora de ejecutarlo.

Podemos simplificar aún más el ejemplo anterior, de la siguiente manera:

In [6]:
fh = open('fear.txt', encoding='utf-8')       # rt es el valor por default
try:
    for line in fh:         # se puede interactuar directamente en fh
        print(line.strip())
finally:
    fh.close()

Sinopsis de LAS INTERMITENCIAS DE LA MUERTE
Autor: JOSÉ SARAMAGO
Una brillante sátira que juega con el miedo más profundo del ser humano, el de perder la vida. En un país cuyo nombre no será mencionado se produce algo nunca visto desde el principio del mundo: la muerte decide suspender su trabajo letal, la gente deja de morir. La eurofia colectiva se desata, pero muy pronto dará paso a la desesperación y al caos. Sobran los motivos. Si es cierto que las personas ya no mueren, eso no significa que el tiempo haya parado. el destino de los humanos será una vejez eterna. Se buscarán maneras de forzar a la muerte a matar aunque no lo quiera, se corromperán las conciencias en los "acuerdos de caballeros" explícitos o tácitos entre el poder político, las mafias y las familias, los ancianos serán detestados ppor haberse convertido en estorbos irremovibles. Hasta el día en que la muerte decide volver...


Como se puede ver, `rt` es el modo por defecto para abrir ficheros, por lo que no necesitamos especificarlo. Además, podemos simplemente iterar sobre `fh`, sin llamar explícitamente a `readlines()` sobre él. Python es muy agradable y nos da atajos para hacer nuestro código más corto y simple de leer.

### Usando un gestor de contexto para abrir archivos
La perspectiva de tener que diseminar nuestro código con bloques `try/finally` no es de las mejores. Como siempre, Python nos da una forma mucho más agradable de abrir un archivo de forma segura: usando un gestor de contexto. Veamos primero el código:

In [7]:
with open('fear.txt', encoding='utf-8') as fh:
    for line in fh:
        print(line.strip())

Sinopsis de LAS INTERMITENCIAS DE LA MUERTE
Autor: JOSÉ SARAMAGO
Una brillante sátira que juega con el miedo más profundo del ser humano, el de perder la vida. En un país cuyo nombre no será mencionado se produce algo nunca visto desde el principio del mundo: la muerte decide suspender su trabajo letal, la gente deja de morir. La eurofia colectiva se desata, pero muy pronto dará paso a la desesperación y al caos. Sobran los motivos. Si es cierto que las personas ya no mueren, eso no significa que el tiempo haya parado. el destino de los humanos será una vejez eterna. Se buscarán maneras de forzar a la muerte a matar aunque no lo quiera, se corromperán las conciencias en los "acuerdos de caballeros" explícitos o tácitos entre el poder político, las mafias y las familias, los ancianos serán detestados ppor haberse convertido en estorbos irremovibles. Hasta el día en que la muerte decide volver...


Este ejemplo es equivalente al anterior, pero se lee mucho mejor. La función `open()` es capaz de producir un objeto fichero cuando es invocada por un gestor de contexto, pero la verdadera belleza reside en que `fh.close()` será llamada automáticamente por nosotros, incluso en caso de error.

## Lectura y escritura en un fichero
Ahora que ya sabemos cómo abrir un archivo, veamos un par de formas distintas de leer y escribir en él:

In [8]:
with open('print_example.txt', 'w') as fw:
    print('Hola, Yo estoy escribiendo dentro de un archivo!!!', file=fw)

Una primera forma es utilizar la función `print()`, que hemos visto muchas veces en las sesiones anteriores. Después de obtener un objeto fichero, esta vez especificando que pretendemos escribir en él `('w')`, podemos decirle a la llamada a `print()` que dirija su salida al fichero, en lugar de al flujo de salida estándar como hace normalmente.

En Python, los flujos estándar de entrada, salida y error están representados por los objetos de archivo `sys.stdin`, `sys.stdout` y `sys.stderr`. A menos que se redirija la entrada o la salida, leer de `sys.stdin` normalmente corresponde a leer del teclado y escribir en `sys.stdout` o `sys.stderr` normalmente imprime en la pantalla de la consola.

Retomando con el código, este tiene el efecto de crear el fichero `print_example.txt` si no existe, o truncarlo en caso afirmativo (truncar un archivo significa borrar su contenido sin eliminarlo; es decir, el archivo sigue existiendo en el sistema de archivos, pero está vacío.), y escribe en él la línea 'Hola, Yo estoy escribiendo dentro de un archivo!!!'.

Todo esto es bonito y fácil, pero no es lo que solemos hacer cuando queremos escribir en un archivo. Veamos un enfoque mucho más común:

In [12]:
with open('fear.txt', encoding='utf-8') as f:
    lines = [line.rstrip() for line in f]

with open('fear_copy.txt', 'w') as fw:
    fw.write('\n'.join(lines))

En este ejemplo, primero abrimos fear.txt y recopilamos su contenido en una lista, línea por línea. Observa que esta vez, estamos llamando a un método diferente, `rstrip()`, como ejemplo, para asegurarnos de que sólo eliminamos los espacios en blanco de la parte derecha de cada línea.

En la segunda parte del fragmento, creamos un nuevo archivo, fear_copy.txt, y escribimos en él todas las líneas del archivo original, unidas por una nueva línea, \n. Python es gracioso y trabaja por defecto con nuevas líneas universales, lo que significa que aunque el archivo original pueda tener una nueva línea diferente a \n, será traducida automáticamente para nosotros antes de que la línea sea devuelta. Este comportamiento es, por supuesto, personalizable, pero normalmente es exactamente lo que quieres.

### Lectura y escritura en modo binario
Observe que al abrir un fichero pasando t en las opciones (u omitiéndola, ya que es la opción por defecto), estamos abriendo el fichero en modo texto. Esto significa que el contenido del fichero se trata e interpreta como texto.

Si desea escribir bytes en un fichero, puede abrirlo en modo binario. Este es un requisito habitual cuando se trabaja con ficheros que no sólo contienen texto en bruto, como imágenes, audio/vídeo y, en general, cualquier otro formato propietario.

Para manejar ficheros en modo binario, basta con especificar la bandera b al abrirlos, como en el siguiente ejemplo:

In [13]:
with open('example.bin', 'wb') as fw:
    fw.write(b'Este es un archivo binario...')

with open('example.bin', 'rb') as f:
    print(f.read())

b'Este es un archivo binario...'


Seguimos utilizando texto como datos binarios, por simplicidad, pero podría ser cualquier cosa que quieras. Puedes ver que se trata como binario por el hecho de que obtienes el prefijo b'Este ...' en la salida.

### Protección contra la sobrescritura de un archivo existente
Python nos ofrece la posibilidad de abrir archivos para escribir en ellos. Usando la bandera $w$, abrimos un archivo y truncamos su contenido. Esto significa que el archivo se sobrescribe con un archivo vacío, y el contenido original se pierde. Si sólo desea abrir un archivo para escribir si aún no existe, puede utilizar la bandera $x$ en su lugar, como en el siguiente ejemplo:

In [18]:
with open('write_x.txt', 'x') as fw: # esto funciona
    fw.write('Escribiendo linea 1')

try:
    with open('write_x.txt', 'x') as fw: # esto falla
        fw.write('Escribiendo linea 2')
except Exception as e:
    print(e)

[Errno 17] File exists: 'write_x.txt'


Al ejecutar el código, encontramos un archivo llamado write_x.txt en su directorio, que contiene una sola línea de texto. De hecho, la segunda parte del fragmento no se ejecuta. Esta es la salida que obtenemos en nuestra consola [Errno 17] File exists: 'write_x.txt'.

## Comprobación de la existencia de archivos y directorios
Si quieres asegurarte de que un archivo o directorio existe (o no), el módulo pathlib es lo que necesitas. Veamos un pequeño ejemplo:

In [8]:
from pathlib import Path
p = Path('fear.txt')
path = p.parent.absolute()
print(f"{p.is_file() = }")
print(f"{path = }")
print(f"{path.is_dir() = }")
q = Path('/Users/rnico/Documents/Proyectos')
print(f"{q.is_dir() = }")

p.is_file() = True
path = WindowsPath('c:/Users/rnico/Documents/Learning_Python/Clase_07_Ficheros_y_Persistencia_de_datos')
path.is_dir() = True
q.is_dir() = True


El fragmento anterior es bastante interesante. Creamos un objeto `Path` que configuramos con el nombre del fichero de texto que queremos inspeccionar. Usamos el método `parent()` para recuperar la carpeta en la que está contenido el fichero, y llamamos al método `absolute()` sobre él, para extraer la información de la ruta absoluta.

Comprobamos si 'fear.txt' es un fichero, y la carpeta en la que está contenido es efectivamente una carpeta (o directorio, que es equivalente).

La forma antigua de realizar estas operaciones era utilizar el módulo os.path de la biblioteca estándar. Mientras que os.path trabaja con cadenas, pathlib ofrece clases que representan rutas del sistema de ficheros con una semántica apropiada para diferentes sistemas operativos. Por lo tanto, sugerimos utilizar pathlib siempre que sea posible, y volver a la antigua forma de hacer las cosas sólo cuando no haya alternativa.

## Manipulando archivos y directorios
Veamos un par de ejemplos rápidos sobre cómo manipular archivos y directorios.

El primer ejemplo manipula el contenido:

In [14]:
from collections import Counter
from string import ascii_letters

chars = ascii_letters + ' '

def sanitize(s, chars):
    return ''.join(c for c in s if c in chars)

def reverse(s):
    return s[::-1]

with open('fear.txt', encoding='utf-8') as stream:
    lines = [line.rstrip() for line in stream]

# escribamos la versión reflejada del archivo
with open('raef.txt', 'w', encoding='utf-8') as stream:
    stream.write('\n'.join(reverse(line) for line in lines))

# ahora podemos calcular algunas estadísticas
lines = [sanitize(line, chars) for line in lines]
whole = ' '.join(lines)

# realizamos comparaciones con la versión minúscula de «whole».
cnt = Counter(whole.lower().split())

# podemos imprimir las N palabras más comunes
print(cnt.most_common(4))

[('la', 8), ('de', 7), ('el', 7), ('las', 5)]


En este ejemplo se han definido dos funciones: `sanitize()` y `reverse()`. Son funciones simples cuyo propósito es eliminar cualquier cosa que no sea una letra o un espacio de una cadena, y producir la copia invertida de una cadena, respectivamente.

Abrimos `fear.txt` y leemos su contenido en una lista. Luego se crea un nuevo fichero, `raef.txt`, que contendrá la versión invertida horizontalmente del original. Escribimos todo el contenido de las líneas con una sola operación, usando `join` sobre un carácter de nueva línea. Quizá sea más interesante la parte del final. Primero, reasignamos las líneas a una versión saneada de sí misma mediante una comprensión de lista. Luego juntamos las líneas en la cadena completa, y finalmente, pasamos el resultado a un objeto `Counter`. Observe que dividimos la versión en minúsculas de la cadena en una lista de palabras. De esta forma, cada palabra será contada correctamente, independientemente de su mayúscula o minúscula, y, gracias a `split()`, no tenemos que preocuparnos de espacios extra en ningún sitio. Cuando imprimimos las cuatras palabras más comunes, nos damos cuenta de que la palabra más común del texto es $"la"$.

Veamos ahora un ejemplo de manipulación más orientado a operaciones de disco, en el que ponemos en uso el módulo `shutil`:

In [15]:
import shutil
from pathlib import Path

base_path = Path('ops_example')

# vamos a realizar una limpieza inicial por si acaso
if base_path.exists() and base_path.is_dir():
    shutil.rmtree(base_path)

# creamos el directorio
base_path.mkdir()
path_b = base_path / 'A' / 'B'
path_c = base_path / 'A' / 'C'
path_d = base_path / 'A' / 'D'
path_b.mkdir(parents=True)
path_c.mkdir()  # ya no son necesarios los padres, puesto que se ha creado «A»

# añadimos tres archivos en 'ops_example/A/B'
for filename in ('ex1.txt', 'ex2.txt', 'ex3.txt'):
    with open(path_b / filename, 'w') as stream:
        stream.write(f'Some content here in {filename}\n')
shutil.move(path_b, path_d)

# también podemos renombrar archivos
ex1 = path_d / 'ex1.txt'
ex1.rename(ex1.parent / 'ex1.renamed.txt')

WindowsPath('ops_example/A/D/ex1.renamed.txt')

En el código anterior, comenzamos declarando una ruta base, que contendrá de forma segura todos los archivos y carpetas que vamos a crear. Luego usamos `mkdir()` para crear dos directorios: `ops_example/A/B` y `ops_example/A/C`. Observa que no necesitamos especificar `parents=True` cuando llamamos a `path_c.mkdir()`, ya que todos los padres ya han sido creados por la llamada anterior en `path_b`.

Usamos el operador `"/"` para concatenar nombres de directorio; `pathlib` se encarga de usar el separador de ruta correcto por nosotros, entre bastidores.

Después de crear los directorios, utilizamos un simple bucle `for` para crear tres ficheros en el directorio B. A continuación, movemos el directorio B y su contenido a un nombre diferente: D. Y finalmente, renombramos **ex1.txt** a **ex1.renamed.txt**. Si abres ese fichero, verás que todavía contiene el texto original de la lógica del bucle `for`.

### Manipulación de nombres de ruta
Exploremos un poco más las capacidades de pathlib mediante un sencillo ejemplo:

In [16]:
from pathlib import Path

p = Path('fear.txt')
print(f"{p.absolute() = }")
print(f"{p.name = }")
print(f"{p.parent.absolute() = }")
print(f"{p.suffix = }")
print(f"{p.parts = }")
print(f"{p.absolute().parts = }")

readme_path = p.parent / '..' / '..' / 'README.rst'
print(f"{readme_path.absolute() = }")
print(f"{readme_path.resolve() = }")

p.absolute() = WindowsPath('c:/Users/rnico/Documents/Learning_Python/Clase_07_Ficheros_y_Persistencia_de_datos/fear.txt')
p.name = 'fear.txt'
p.parent.absolute() = WindowsPath('c:/Users/rnico/Documents/Learning_Python/Clase_07_Ficheros_y_Persistencia_de_datos')
p.suffix = '.txt'
p.parts = ('fear.txt',)
p.absolute().parts = ('c:\\', 'Users', 'rnico', 'Documents', 'Learning_Python', 'Clase_07_Ficheros_y_Persistencia_de_datos', 'fear.txt')
readme_path.absolute() = WindowsPath('c:/Users/rnico/Documents/Learning_Python/Clase_07_Ficheros_y_Persistencia_de_datos/../../README.rst')
readme_path.resolve() = WindowsPath('C:/Users/rnico/Documents/README.rst')


Observe cómo, en las dos últimas líneas, tenemos dos representaciones diferentes de la misma ruta. La primera (`readme_path.absolute()`) muestra dos `'..'`, una sola de las cuales, en términos de ruta, indica el cambio a la carpeta padre. Así, cambiando a la carpeta padre dos veces seguidas, de .../Documents/Learning_Python/ volvemos a .../Documents/. Esto se confirma en la última línea del ejemplo, que muestra la salida de readme_path.resolve().

## Archivos y directorios temporales
A veces, es muy útil poder crear un directorio o archivo temporal al ejecutar algún código. Por ejemplo, cuando escribes pruebas que afectan al disco, puedes utilizar ficheros y directorios temporales para ejecutar tu lógica y afirmar que es correcta, y para asegurarte de que al final de la ejecución de la prueba, la carpeta de pruebas no tiene restos. Veamos cómo hacerlo en Python:

In [18]:
from tempfile import NamedTemporaryFile, TemporaryDirectory

with TemporaryDirectory(dir='.') as td:
    print('Temp directory:', td)
    with NamedTemporaryFile(dir=td) as t:
        name = t.name
        print(name)

Temp directory: c:\Users\rnico\Documents\Learning_Python\Clase_07_Ficheros_y_Persistencia_de_datos\tmp5pctr8q2
c:\Users\rnico\Documents\Learning_Python\Clase_07_Ficheros_y_Persistencia_de_datos\tmp5pctr8q2\tmpwech_qbd


El ejemplo anterior es bastante sencillo: creamos un directorio temporal en el actual («.»), y creamos en él un fichero temporal con nombre. Imprimimos el nombre del fichero, así como su ruta completa; además, tener en cuenta que la ejecución de este script producirá un resultado diferente cada vez.

## Contenido del directorio
Con Python, también puedes inspeccionar el contenido de un directorio. Te mostraremos dos formas de hacerlo. Esta es la primera:

In [19]:
from pathlib import Path
p = Path('.')
for entry in p.glob('*'):
    print('File:' if entry.is_file() else 'Folder:', entry)

File: example.bin
File: fear.txt
File: fear_copy.txt
Folder: ops_example
File: print_example.txt
File: raef.txt
File: Sesion_07.ipynb
File: write_x.txt


Este fragmento utiliza el método `glob()` de un objeto `Path`, aplicado desde el directorio actual. Iteramos sobre los resultados, cada uno de los cuales es una instancia de una subclase de `Path` (`PosixPath` o `WindowsPath`, según el Sistema Operativo que estemos ejecutando). Para cada entrada, inspeccionamos si es un directorio, e imprimimos el resultado.

Una forma alternativa de escanear un árbol de directorios es mediante `os.walk`. Veamos un ejemplo:

In [20]:
import os

for root, dirs, files in os.walk('.'):
    abs_root = os.path.abspath(root)
    print(abs_root)

    if dirs:
        print('Directories:')
        for dir_ in dirs:
            print(dir_)
        print()

    if files:
        print('Files:')
        for filename in files:
            print(filename)
        print()

c:\Users\rnico\Documents\Learning_Python\Clase_07_Ficheros_y_Persistencia_de_datos
Directories:
ops_example

Files:
example.bin
fear.txt
fear_copy.txt
print_example.txt
raef.txt
Sesion_07.ipynb
write_x.txt

c:\Users\rnico\Documents\Learning_Python\Clase_07_Ficheros_y_Persistencia_de_datos\ops_example
Directories:
A

c:\Users\rnico\Documents\Learning_Python\Clase_07_Ficheros_y_Persistencia_de_datos\ops_example\A
Directories:
C
D

c:\Users\rnico\Documents\Learning_Python\Clase_07_Ficheros_y_Persistencia_de_datos\ops_example\A\C
c:\Users\rnico\Documents\Learning_Python\Clase_07_Ficheros_y_Persistencia_de_datos\ops_example\A\D
Files:
ex1.renamed.txt
ex2.txt
ex3.txt



La ejecución del fragmento anterior producirá una lista de todos los archivos y directorios del actual, y hará lo mismo para cada subdirectorio.

## Compresión de archivos y directorios
Veamos un ejemplo de cómo crear un archivo comprimido. Python te permite crear archivos comprimidos de varias formas y formatos diferentes. Aquí, vamos a mostrarte cómo crear el más común, ZIP:

In [22]:
from zipfile import ZipFile
import os

# Crear la carpeta 'subfolder' si no existe
if not os.path.exists('subfolder'):
    os.makedirs('subfolder')

# Archivos de prueba
with open('content1.txt', 'w') as f:
    f.write("Este es el contenido del archivo 1.")

with open('content2.txt', 'w') as f:
    f.write("Este es el contenido del archivo 2.")

# Crear los archivos en la carpeta 'subfolder'
with open('subfolder/content3.txt', 'w') as f:
    f.write("Este es el contenido del archivo 3 en la subcarpeta.")

with open('subfolder/content4.txt', 'w') as f:
    f.write("Este es el contenido del archivo 4 en la subcarpeta.")


with ZipFile('example.zip', 'w') as zp:
    zp.write('content1.txt')
    zp.write('content2.txt')
    zp.write('subfolder/content3.txt')
    zp.write('subfolder/content4.txt')

with ZipFile('example.zip') as zp:
    zp.extract('content1.txt', 'extract_zip')
    zp.extract('subfolder/content3.txt', 'extract_zip')

En el código anterior, importamos `ZipFile` y, luego, dentro de un gestor de contexto, escribimos en él cuatro archivos (dos de los cuales están en una subcarpeta, para mostrar cómo ZIP conserva la ruta completa). Después, a modo de ejemplo, abrimos el archivo comprimido y extraemos de él un par de ficheros al directorio `extract_zip`.

# Formatos de intercambio de datos
La arquitectura de software moderna tiende a dividir una aplicación en varios componentes. Tanto si adopta el paradigma de la arquitectura orientada a servicios como si lo lleva aún más lejos en el ámbito de los microservicios, estos componentes tendrán que intercambiar datos. Pero incluso si está codificando una aplicación monolítica cuya base de código está contenida en un proyecto, lo más probable es que todavía tenga que intercambiar datos con API, otros programas, o simplemente manejar el flujo de datos entre las partes frontend y backend de su sitio web, que muy probablemente no hablen el mismo idioma.

Elegir el formato adecuado para intercambiar información es crucial. Un formato específico del lenguaje tiene la ventaja de que el lenguaje en sí es muy probable que le proporcione todas las herramientas para hacer la serialización y deserialización de una brisa. Sin embargo, perderás la capacidad de comunicarte con otros componentes que hayan sido escritos en versiones diferentes del mismo lenguaje, o en lenguajes completamente diferentes. Independientemente de cómo sea el futuro, sólo se debería optar por un formato específico para un idioma si es la única opción posible para una situación determinada.

Según Wikipedia (https://en.wikipedia.org/wiki/Serialization):
En informática, la serialización es el proceso de traducir una estructura de datos o el estado de un objeto a un formato que pueda almacenarse (por ejemplo, en un archivo o en un búfer de datos de memoria) o transmitirse (por ejemplo, a través de una red informática) y reconstruirse más tarde (posiblemente en un entorno informático diferente).

Un enfoque mucho mejor es elegir un formato que sea agnóstico con respecto al idioma y que pueda ser utilizado por todos los idiomas (o al menos por la mayoría). 

En el mundo del software, algunos formatos populares se han convertido en la norma de facto para el intercambio de datos. Los más famosos son probablemente XML, YAML y JSON. La biblioteca estándar de Python incluye los módulos xml y json y, en PyPI (https://pypi.org/), puedes encontrar algunos paquetes diferentes para trabajar con YAML. 

En el entorno Python, JSON es quizás el más utilizado. Gana a los otros dos por ser parte de la librería estándar, y por su simplicidad. Si alguna vez has trabajado con XML, sabes la pesadilla que puede llegar a ser.

Además, cuando se trabaja con una base de datos como PostgreSQL, la posibilidad de utilizar campos JSON nativos constituye un argumento de peso para adoptar también JSON en la aplicación.

### Trabajando con JSON
JSON es el acrónimo de JavaScript Object Notation, y es un subconjunto del lenguaje JavaScript. Existe desde hace casi dos décadas, por lo que es bien conocido y ampliamente adoptado por la mayoría de los lenguajes, aunque en realidad es independiente del lenguaje. Puedes leer todo sobre él en su página web (https://www.json.org/).

JSON se basa en dos estructuras: una colección de pares nombre/valor y una lista ordenada de valores. Es bastante sencillo darse cuenta de que estos dos objetos son similares con los tipos de datos dict y list de Python, respectivamente. Como tipos de datos, JSON ofrece cadenas, números, objetos y valores que consisten en verdadero, falso y nulo. Veamos un ejemplo rápido para empezar:

In [1]:
import sys
import json

data = {
    'big_number': 2 ** 3141,
    'max_float': sys.float_info.max,
    'a_list': [2, 3, 5, 7],
}

json_data = json.dumps(data)
data_out = json.loads(json_data)
assert data == data_out # json and back, data matches

Comenzamos importando los módulos `sys` y `json`. Luego creamos un diccionario simple con algunos valores númericos y una lista. En este ejemplo, estamos probando la serialización y deserialización usando números muy grandes, tanto int como float, así que ponemos 23141 y lo que sea el mayor número en coma flotante que nuestro sistema pueda manejar.

Serializamos con `json.dumps()`, que toma los datos y los convierte en una cadena con formato `JSON`. Esos datos se introducen en `json.loads()`, que hace lo contrario: a partir de una cadena con formato `JSON`, reconstruye los datos en Python. En la última línea, nos aseguramos de que los datos originales y el resultado de la serialización/deserialización a través de JSON coinciden.

Veamos qué aspecto tendrían los datos `JSON` si los imprimiéramos:

In [6]:
import json
info = {
    'full_name': 'Sherlock Holmes',
    'address': {
        'street': '221B Baker St',
        'zip': 'NW1 6XE',
        'city': 'London',
        'country': 'UK',
    }
}
print(json.dumps(info, indent=2, sort_keys=True))

{
  "address": {
    "city": "London",
    "country": "UK",
    "street": "221B Baker St",
    "zip": "NW1 6XE"
  },
  "full_name": "Sherlock Holmes"
}


En este ejemplo, creamos un diccionario con los datos de Sherlock Holmes. Además, nos podemos dar cuenta de cómo llamamos a `json.dumps`. Le hemos dicho que haga sangría con dos espacios, y que ordene las claves alfabéticamente.

La similitud con Python es evidente. La única diferencia es que si colocas una coma en el último elemento de un diccionario, como es habitual en Python, JSON se quejará.

Veamos algo interesante:

In [7]:
import json
data_in = {
    'a_tuple': (1, 2, 3, 4, 5),
}
json_data = json.dumps(data_in)
print(f"{json_data = }")
data_out = json.loads(json_data)
print(f"{data_out = }")

json_data = '{"a_tuple": [1, 2, 3, 4, 5]}'
data_out = {'a_tuple': [1, 2, 3, 4, 5]}


En este ejemplo, hemos utilizado una tupla en lugar de una lista. Lo interesante es que, conceptualmente, una tupla es también una lista ordenada de elementos. No tiene la flexibilidad de una lista, pero aún así, se considera lo mismo desde la perspectiva de JSON. Por lo tanto, como se puede ver por la primera impresión, en JSON una tupla se transforma en una lista. Naturalmente entonces, la información de que el objeto original era una tupla se pierde, y cuando ocurre la deserialización, `a_tupla` es en realidad traducida a una lista Python. Es importante que tengas esto en cuenta cuando trates con datos, ya que pasar por un proceso de transformación que implica un formato que sólo comprende un subconjunto de las estructuras de datos que puedes usar implica que puede haber pérdida de información. En este caso, hemos perdido la información sobre el tipo (tupla frente a lista).

En realidad, se trata de un problema habitual. Por ejemplo, no puedes serializar todos los objetos Python a JSON, ya que no siempre está claro cómo JSON debe revertir ese objeto. Piensa en `datetime`, por ejemplo. Una instancia de esa clase es un objeto Python que JSON no podrá serializar. Si lo transformamos en una cadena como $2018-03-04T12:00:30Z$, que es la representación ISO 8601 de una fecha con información de hora y zona horaria, JSON tendría limitaciones ya que cuando se trata de intercambio de datos, a menudo necesitamos transformar nuestros objetos a un formato más simple antes de serializarlos con JSON. Cuanto más consigamos simplificar nuestros datos, más fácil será representarlos en un formato como JSON.

En algunos casos, sin embargo, y sobre todo para uso interno, es útil poder serializar objetos personalizados, así que vamos a mostrarte cómo con dos ejemplos: números complejos y objetos datetime.

#### Codificación/decodificación personalizada con JSON
En el mundo JSON, podemos considerar términos como codificación/decodificación como sinónimos de serialización/deserialización. En el siguiente ejemplo, vamos a aprender cómo codificar números complejos - que no son serializables a JSON por defecto - escribiendo un codificador personalizado:

In [9]:
import json

class ComplexEncoder(json.JSONEncoder):
    def default(self, obj):
        
        print(f"ComplexEncoder.default: {obj=}")
        if isinstance(obj, complex):
            return {
                '_meta': '_complex',
                'num': [obj.real, obj.imag],
            }
        return super().default(obj)

data = {
    'an_int': 42,
    'a_float': 3.14159265,
    'a_complex': 3 + 4j,
}

json_data = json.dumps(data, cls=ComplexEncoder)
print(f"{json_data = }")

def object_hook(obj):
    print(f"object_hook: {obj=}")
    try:
        if obj['_meta'] == '_complex':
            return complex(*obj['num'])
    except KeyError:
        return obj

data_out = json.loads(json_data, object_hook=object_hook)
print(f"{data_out = }")

ComplexEncoder.default: obj=(3+4j)
json_data = '{"an_int": 42, "a_float": 3.14159265, "a_complex": {"_meta": "_complex", "num": [3.0, 4.0]}}'
object_hook: obj={'_meta': '_complex', 'num': [3.0, 4.0]}
object_hook: obj={'an_int': 42, 'a_float': 3.14159265, 'a_complex': (3+4j)}
data_out = {'an_int': 42, 'a_float': 3.14159265, 'a_complex': (3+4j)}


Comenzamos definiendo una clase `ComplexEncoder` como subclase de la clase `JSONEncoder`. Nuestra clase sobrescribe el método por defecto. Este método es llamado cada vez que el codificador encuentra un objeto que no puede codificar y se espera que devuelva una representación codificable de ese objeto.

Nuestro método `default()` comprueba si su argumento es un objeto complejo, en cuyo caso devuelve un diccionario con alguna meta información personalizada, y una lista que contiene tanto la parte real como la imaginaria del número. Eso es todo lo que tenemos que hacer para evitar perder la información de un número complejo. Si recibimos cualquier cosa que no sea una instancia de complex, llamamos al método `default()` de la clase padre, que simplemente lanza un `TypeError`. A continuación llamamos a `json.dumps()`, pero esta vez utilizamos el argumento `cls` para especificar nuestro codificador personalizado.

La mitad del trabajo está hecho. Para la parte de deserialización, podríamos haber escrito otra clase que heredara de `JSONDecoder`, pero en su lugar hemos optado por utilizar una técnica diferente que es más sencilla y utiliza una pequeña función: `object_hook`.

Dentro del cuerpo de `object_hook()`, encontramos un bloque `try`. La parte importante son las dos líneas dentro del cuerpo del propio bloque `try`. La función recibe un objeto (observe que la función sólo se llama cuando `obj` es un diccionario), y si los metadatos coinciden con nuestra convención para números complejos, pasamos las partes real e imaginaria a la función `complex()`. El bloque `try/except` está ahí porque nuestra función será llamada para cada objeto diccionario que sea decodificado, así que necesitamos manejar el caso en el que nuestra clave _meta no esté presente. 

Puedes ver que `a_complex` se ha deserializado correctamente en el ejercicio anterior.

Consideremos ahora un ejemplo un poco más complejo: tratar con objetos `datetime`. Vamos a dividir el código en dos bloques, primero la parte de serialización y luego la de deserialización:

In [10]:
import json
from datetime import datetime, timedelta, timezone

now = datetime.now()
now_tz = datetime.now(tz=timezone(timedelta(hours=1)))

class DatetimeEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, datetime):
            try:
                off = obj.utcoffset().seconds
            except AttributeError:
                off = None
            
            return {
                '_meta': '_datetime',
                'data': obj.timetuple()[:6] + (obj.microsecond, ),
                'utcoffset': off,
            }
        
        return super().default(obj)

data = {
    'an_int': 42,
    'a_float': 3.14159265,
    'a_datetime': now,
    'a_datetime_tz': now_tz,
}

json_data = json.dumps(data, cls=DatetimeEncoder)
print(f"{json_data = }")

json_data = '{"an_int": 42, "a_float": 3.14159265, "a_datetime": {"_meta": "_datetime", "data": [2024, 9, 14, 18, 46, 31, 903758], "utcoffset": null}, "a_datetime_tz": {"_meta": "_datetime", "data": [2024, 9, 15, 0, 46, 31, 903758], "utcoffset": 3600}}'


La razón por la que este ejemplo es ligeramente más complejo reside en el hecho de que los objetos `datetime` en Python pueden tener en cuenta la zona horaria o no; por lo tanto, tenemos que ser más cuidadosos. El flujo es el mismo que antes, sólo que estamos tratando con un tipo de datos diferente. Comenzamos obteniendo la información de la fecha y hora actuales, y lo hacemos tanto sin (`now`) como con (`now_tz`) conocimiento de la zona horaria, sólo para asegurarnos de que nuestro script funciona. Luego procedemos a definir un codificador personalizado como antes, sobrescribiendo el método `default()`. Lo importante en ese método es cómo obtenemos la información de la zona horaria (off), en segundos, y cómo estructuramos el diccionario que devuelve los datos. Esta vez, los metadatos dicen que es información de fecha y hora. Guardamos los seis primeros elementos de la tupla de tiempo (año, mes, día, hora, minuto y segundo), más los microsegundos en la clave de datos, y el desfase después de eso.

Cuando tenemos nuestro codificador personalizado, procedemos a crear algunos datos, y luego serializamos.

Curiosamente, descubrimos que `None` se traduce a `null`, su equivalente en JavaScript. Además, podemos ver que nuestros datos parecen haber sido codificados correctamente. Procedamos con la segunda parte del script:

In [12]:
def object_hook(obj):
    try:
        if obj['_meta'] == '_datetime':
            if obj['utcoffset'] is None:
                tz = None
            else:
                tz = timezone(timedelta(seconds=obj['utcoffset']))
            return datetime(*obj['data'], tzinfo=tz)
    except KeyError:
        return obj
data_out = json.loads(json_data, object_hook=object_hook)
print(f"{data_out = }")

data_out = {'an_int': 42, 'a_float': 3.14159265, 'a_datetime': datetime.datetime(2024, 9, 14, 18, 46, 31, 903758), 'a_datetime_tz': datetime.datetime(2024, 9, 15, 0, 46, 31, 903758, tzinfo=datetime.timezone(datetime.timedelta(seconds=3600)))}


Una vez más, primero verificamos que los metadatos nos indican que se trata de una fecha y hora y, a continuación, obtenemos la información sobre la zona horaria. Una vez que tenemos eso, pasamos la tupla (usando * para descomprimir sus valores en la llamada) y la información de la zona horaria a la llamada `datetime()`, obteniendo de vuelta nuestro objeto original.

# I/O, flujos y peticiones
I/O son las siglas de input/output, y en términos generales se refiere a la comunicación entre un ordenador y el mundo exterior. Hay varios tipos diferentes de I/O, y está fuera del alcance de esta sesión explicarlos todos, pero vale la pena repasar un par de ejemplos. El primero introducirá la clase `io.StringIO`, que es un flujo en memoria para I/O de texto. El segundo en cambio escapará a la localidad de nuestro ordenador, y demostrará cómo realizar una petición HTTP.

## Utilización de un flujo en memoria
Los objetos en memoria pueden ser útiles en multitud de situaciones. La memoria es mucho más rápida que un disco, siempre está disponible, y para pequeñas cantidades de datos puede ser la elección perfecta.

Veamos el primer ejemplo:

In [15]:
import io
stream = io.StringIO()
stream.write('Aprendiendo a programar en Python.\n')
print('Conviértete en un ninja de Python!', file=stream)
contents = stream.getvalue()
print(contents)
stream.close()

Aprendiendo a programar en Python.
Conviértete en un ninja de Python!



En el fragmento de código anterior, importamos el módulo `io` de la biblioteca estándar. Este es un módulo muy interesante que cuenta con muchas herramientas relacionadas con flujos e I/O. Una de ellas es `StringIO`, que es un buffer en memoria en el que vamos a escribir dos sentencias, utilizando dos métodos diferentes. Podemos llamar a `StringIO.write()` o podemos usar `print`, indicándole que dirija los datos a nuestro stream.

Llamando a `getvalue()`, podemos obtener el contenido del `stream`. Luego procedemos a imprimirlo, y finalmente lo cerramos. La llamada a `close()` hace que el buffer de texto sea descartado inmediatamente.

Hay una forma más elegante de escribir el código anterior usando un gestor de contexto:

In [17]:
with io.StringIO() as stream:
    stream.write('Aprendiendo a programar en Python.\n')
    print('Conviértete en un ninja de Python!', file=stream)
    contents = stream.getvalue()
    print(contents)

Aprendiendo a programar en Python.
Conviértete en un ninja de Python!



## Realizar peticiones HTTP
En esta sección, exploraremos dos ejemplos sobre peticiones HTTP. Usaremos la librería `requests` para estos ejemplos, la cual puedes instalar con pip.

Vamos a realizar peticiones HTTP a la API httpbin.org (http://httpbin.org/), que, curiosamente, fue desarrollada por Kenneth Reitz, el creador de la propia librería requests.

Esta librería está entre las más adoptadas en todo el mundo:

In [19]:
import requests

urls = {
    "get": "https://httpbin.org/get?t=learn+python+programming",
    "headers": "https://httpbin.org/headers",
    "ip": "https://httpbin.org/ip",
    "user-agent": "https://httpbin.org/user-agent",
    "UUID": "https://httpbin.org/uuid",
    "JSON": "https://httpbin.org/json",
}

def get_content(title, url):
    resp = requests.get(url)
    print(f"Response for {title}")
    print(resp.json())

for title, url in urls.items():
    get_content(title, url)
    print("-" * 40)

Response for get
{'args': {'t': 'learn python programming'}, 'headers': {'Accept': '*/*', 'Accept-Encoding': 'gzip, deflate', 'Host': 'httpbin.org', 'User-Agent': 'python-requests/2.32.3', 'X-Amzn-Trace-Id': 'Root=1-66e626a2-2f7ebd1431e75d7e2e41270a'}, 'origin': '38.250.152.202', 'url': 'https://httpbin.org/get?t=learn+python+programming'}
----------------------------------------
Response for headers
{'headers': {'Accept': '*/*', 'Accept-Encoding': 'gzip, deflate', 'Host': 'httpbin.org', 'User-Agent': 'python-requests/2.32.3', 'X-Amzn-Trace-Id': 'Root=1-66e626a2-42b1ffaf416c747f37be6f5b'}}
----------------------------------------
Response for ip
{'origin': '38.250.152.202'}
----------------------------------------
Response for user-agent
{'user-agent': 'python-requests/2.32.3'}
----------------------------------------
Response for UUID
{'uuid': 'a8bf6799-c55d-4150-9e11-ec5791f96f0d'}
----------------------------------------
Response for JSON
{'slideshow': {'author': 'Yours Truly', 'dat

El fragmento anterior debería ser fácil de entender. Declaramos un diccionario de URLs a las que queremos realizar peticiones HTTP. Se ha encapsulado el código que realiza la petición en una pequeña función, `get_content()`. Como puedes ver, realizamos una petición GET (usando `requests.get()`), e imprimimos el título y la versión decodificada JSON del cuerpo de la respuesta.

Cuando realizamos una solicitud a un sitio web, o a una API, obtenemos un objeto de respuesta, que es, simplemente, lo que nos devuelve el servidor al que hemos realizado la solicitud. El cuerpo de algunas respuestas de httpbin.org está codificado en JSON, así que en lugar de obtener el cuerpo tal cual (usando resp.text) y decodificarlo manualmente llamando a `json.loads()`, simplemente combinamos los dos métodos utilizando `json()` en el objeto de respuesta. Hay muchas razones por las que el paquete `requests` ha sido tan ampliamente adoptado, y una de ellas es sin duda su facilidad de uso.

Al final del código, ejecutamos un bucle `for` y obtenemos todas las URLs. Cuando lo ejecutamos, vemos el resultado de cada llamada impreso en la consola.

Tenga en cuenta que puede obtener un resultado ligeramente diferente en términos de números de versión e IPs, lo cual está bien si lo corre en otra computadora y en otro momento. Ahora, GET es sólo uno de los verbos HTTP, aunque uno de los más utilizados. Veamos también cómo utilizar el verbo POST. Este es el tipo de petición que haces cuando necesitas enviar datos al servidor. Cada vez que envías un formulario en la web, estás haciendo una petición POST. Así que, veamos un ejemplo:

In [20]:
import requests

url = 'https://httpbin.org/post'
data = dict(title='Learn Python Programming')
resp = requests.post(url, data=data)
print('Response for POST')
print(resp.json())

Response for POST
{'args': {}, 'data': '', 'files': {}, 'form': {'title': 'Learn Python Programming'}, 'headers': {'Accept': '*/*', 'Accept-Encoding': 'gzip, deflate', 'Content-Length': '30', 'Content-Type': 'application/x-www-form-urlencoded', 'Host': 'httpbin.org', 'User-Agent': 'python-requests/2.32.3', 'X-Amzn-Trace-Id': 'Root=1-66e62857-6fe5e4887caf3ae171008852'}, 'json': None, 'origin': '38.250.152.202', 'url': 'https://httpbin.org/post'}


El código anterior es muy similar al que vimos antes, sólo que esta vez no llamamos a `get()`, sino a `post()`, y como queremos enviar algunos datos, lo especificamos en la llamada. La librería `requests` ofrece mucho más que esto.

Observa que la salida del fragmento de código tiene las cabeceras diferentes, y encontramos los datos que enviamos en el par clave/valor del formulario del cuerpo de la respuesta.

# Persistencia de datos en disco
En esta última sección de este capítulo, veremos cómo persistir datos en disco en tres formatos diferentes. Persistir datos significa que los datos se escriben en un almacenamiento no volátil, como un disco duro, por ejemplo, y no se borran cuando el proceso que los escribió termina su ciclo de vida. Exploraremos `pickle` y `shelve`, así como un breve ejemplo que implicará acceder a una base de datos usando `SQLAlchemy`, quizás la librería ORM más ampliamente adoptada en el ecosistema Python.

## Serialización de datos con pickle
El módulo `pickle`, de la biblioteca estándar de Python, ofrece herramientas para convertir objetos Python en flujos de bytes, y viceversa. Aunque hay un solapamiento parcial en la API que pickle y json exponen, los dos son bastante diferentes. Como hemos visto anteriormente en este capítulo, JSON es un formato de texto legible por humanos, independiente del lenguaje y que sólo soporta un subconjunto restringido de tipos de datos de Python. El módulo `pickle`, por otro lado, no es legible por humanos, se traduce a bytes, es específico de Python y, gracias a las maravillosas capacidades de introspección de Python, soporta un gran número de tipos de datos.

Además de las diferencias mencionadas entre pickle y json, también hay algunos problemas de seguridad importantes que debes tener en cuenta si estás considerando usar pickle. Unpickle datos erróneos o maliciosos de una fuente no confiable puede ser peligroso, por lo que si decidimos adoptarlo en nuestra aplicación, tenemos que tener mucho cuidado.

Dicho esto, veámoslo en acción mediante un sencillo ejemplo:

In [23]:
import pickle
from dataclasses import dataclass

@dataclass
class Person:
    first_name: str
    last_name: str
    id: int

    def greet(self):
        print(f'Hola, Yo soy {self.first_name} {self.last_name}'
              f' y mi ID es {self.id}')

people = [
    Person('Nicolas', 'Marroquin', 123),
    Person('Jairo', 'Rubio', 456),
]

# guardar datos en formato binario en un archivo
with open('data.pickle', 'wb') as stream:
    pickle.dump(people, stream)

# cargar datos de un archivo
with open('data.pickle', 'rb') as stream:
    peeps = pickle.load(stream)

for person in peeps:
    person.greet()

Hola, Yo soy Nicolas Marroquin y mi ID es 123
Hola, Yo soy Jairo Rubio y mi ID es 456


En este ejemplo, creamos una clase Persona utilizando el decorador dataclass. La única razón por la que escribimos este ejemplo con una clase de datos es para mostrarte la facilidad con la que pickle trata con ella, sin necesidad de que hagamos nada que no haríamos para un tipo de datos más simple.

La clase tiene tres atributos: nombre, apellido e id. También expone un método `greet()`, que simplemente imprime un mensaje hola con los datos.

Creamos una lista de instancias y la guardamos en un fichero. Para ello, utilizamos `pickle.dump()`, a la que introducimos el contenido a recoger, y el `stream` en el que queremos escribir. Inmediatamente después, leemos desde ese mismo fichero, usando `pickle.load()` para convertir todo el contenido del flujo de nuevo en objetos Python. Para asegurarnos de que los objetos se han convertido correctamente, llamamos al método `greet()` en ambos casos.

El módulo `pickle` también permite convertir a (y desde) objetos byte, mediante las funciones `dumps()` y `loads()` (nótese la $s$ al final de ambos nombres). En las aplicaciones del día a día, `pickle` se suele utilizar cuando necesitamos persistir datos de Python que se supone que no se van a intercambiar con otra aplicación.

Otra herramienta que posiblemente se utiliza aún menos, pero que resulta ser muy útil cuando estás corto de recursos, es `shelve`.

## Guardar datos con shelve
`Shelve` es un objeto persistente similar a un diccionario. Lo bueno de esto es que los valores que guardas en un `shelf` pueden ser cualquier objeto que puedas seleccionar, así que no estás restringido como lo estarías si estuvieras usando una base de datos. Aunque interesante y útil, el módulo `shelve` se utiliza muy poco en la práctica. Sólo para completar, veamos un ejemplo rápido de cómo funciona:

In [25]:
import shelve

class Person:
    def __init__(self, name, id):
        self.name = name
        self.id = id

with shelve.open('shelf1.shelve') as db:
    db['obi1'] = Person('Obi-Wan', 123)
    db['ani'] = Person('Anakin', 456)
    db['a_list'] = [2, 3, 5]
    db['delete_me'] = 'we will have to delete this one...'
    print(f"{list(db.keys()) = }")
    
    del db['delete_me']
    print(f"{list(db.keys()) = }")
    print(f"{'delete_me' in db = }")
    print(f"{'ani' in db = }")

    a_list = db['a_list']
    a_list.append(7)
    db['a_list'] = a_list
    print(f"{db['a_list'] = }")

list(db.keys()) = ['obi1', 'ani', 'a_list', 'delete_me']
list(db.keys()) = ['obi1', 'ani', 'a_list']
'delete_me' in db = False
'ani' in db = True
db['a_list'] = [2, 3, 5, 7]


Este ejemplo se parece a un ejercicio con diccionarios. Creamos una simple clase `Person` y luego abrimos un archivo de `shelve` dentro de un gestor de contexto. Como puedes ver, utilizamos la sintaxis de diccionario para almacenar cuatro objetos: dos instancias de `Person`, una lista y una cadena. Si imprimimos las claves, obtenemos una lista que contiene las cuatro claves que hemos utilizado. Inmediatamente después de imprimirla, borramos el par clave/valor (acertadamente llamado) delete_me del `shelf`. Si volvemos a imprimir las claves, veremos que la eliminación se ha realizado correctamente. A continuación, comprobamos la pertenencia de un par de claves y, por último, añadimos el número 7 a `a_list`. Observa cómo tenemos que extraer la lista del `shelf`, modificarla y guardarla de nuevo.

Si este comportamiento no es deseado, hay algo que podemos hacer:

In [26]:
with shelve.open('shelf2.shelve', writeback=True) as db:
    db['a_list'] = [11, 13, 17]
    db['a_list'].append(19)
    print(f"{db['a_list'] = }")

db['a_list'] = [11, 13, 17, 19]


Al abrir el `shelf` con `writeback=True`, habilitamos la función de reescritura, que nos permite simplemente agregar a `a_list` como si realmente fuera un valor dentro de un diccionario normal. La razón por la que esta función no está activa de forma predeterminada es que tiene un precio que pagas en términos de consumo de memoria y cierre más lento de la estantería.

Ahora que hemos rendido homenaje a los módulos de biblioteca estándar relacionados con la persistencia de datos, echemos un vistazo a uno de los ORM más adoptados en el ecosistema Python: `SQLAlchemy`.

## Guardar datos en una base de datos
Para este ejemplo, vamos a trabajar con una base de datos en memoria, lo que nos simplificará las cosas.

Antes de sumergirnos en el código, permítanos introducir brevemente el concepto de una base de datos relacional.

Una base de datos relacional es una base de datos que le permite guardar datos siguiendo el modelo relacional, inventado en 1969 por Edgar F. Codd. En este modelo, los datos se almacenan en una o más tablas. Cada tabla tiene filas (también conocidas como registros, o tuplas), cada una de las cuales representa una entrada en la tabla. Las tablas también tienen columnas (también conocidas como atributos), cada una de las cuales representa un atributo de los registros. Cada registro se identifica a través de una clave única (unique key), más comúnmente conocida como la clave principal (primary key), que es la unión de una o más columnas en la tabla. Para darle un ejemplo: imagine una tabla llamada usuarios, con ID de columnas, nombre de usuario, contraseña, nombre y apellido.

Una tabla de este tipo sería perfecta para contener a los usuarios de nuestro sistema. Cada fila representaría a un usuario diferente. Por ejemplo, una fila con los valores 3, fab, my_wonderful_pwd, Fabrizio y Romano representaría al usuario de Fabrizio en el sistema.

La razón por la que el modelo se llama relacional es porque se pueden establecer relaciones entre tablas. Por ejemplo, si se agrega una tabla llamada PhoneNumbers a nuestra base de datos ficticia, podría insertar números de teléfono en ella y, a continuación, a través de una relación, establecer qué número de teléfono pertenece a qué usuario.

Para consultar una base de datos relacional, necesitamos un lenguaje especial. El estándar principal se llama SQL, que significa Structured Query Language (Lenguaje de Consulta Estructurado). Nace de algo llamado álgebra relacional, que es una familia de álgebras que se utilizan para modelar datos almacenados de acuerdo con el modelo relacional y realizar consultas sobre ellos. Las operaciones más comunes que se pueden realizar suelen consistir en filtrar por filas o columnas, unir tablas, agregar los resultados según algunos criterios, etc. Para darte un ejemplo, una consulta en nuestra base de datos imaginaria podría ser: Obtener todos los usuarios (nombre de usuario, nombre, apellido) cuyo nombre de usuario comience con "m", que tengan como máximo un número de teléfono. En esta consulta, solicitamos un subconjunto de las columnas de la tabla Usuario. Estamos filtrando a los usuarios tomando solo aquellos cuyo nombre de usuario comienza con la letra m, e incluso más allá, solo aquellos que tienen como máximo un número de teléfono.

Ahora, cada base de datos viene con su propio sabor de SQL. Todos respetan la norma hasta cierto punto, pero ninguno lo hace plenamente, y todos son diferentes entre sí en algunos aspectos. Esto plantea un problema en el desarrollo de software moderno. Si nuestra aplicación contiene código SQL, es bastante probable que si decidimos usar un motor de base de datos diferente, o tal vez una versión diferente del mismo motor, encontraríamos que nuestro código SQL necesita ser modificado.

Esto puede ser bastante doloroso, especialmente porque las consultas SQL pueden complicarse con bastante rapidez. Para aliviar un poco este dolor, los informáticos han creado un código que mapea objetos de un lenguaje de programación a tablas de una base de datos relacional. Como era de esperar, el nombre de dicha herramienta es Object-Relational Mapping (ORM) que en español se traduce como Mapeo Relacional de Objetos.

En el desarrollo de aplicaciones modernas, normalmente comenzaría a interactuar con una base de datos mediante el uso de un ORM, y si se encuentra en una situación en la que no puede realizar una consulta que necesita realizar a través del ORM, recurriría al uso de SQL directamente. Este es un buen compromiso entre no tener SQL en absoluto y no usar ORM, lo que en última instancia significa especializar el código que interactúa con la base de datos, con las desventajas antes mencionadas.

En esta sección, vamos mostrar un ejemplo que aprovecha `SQLAlchemy`, uno de los ORM de Python de terceros más populares. Para ello, se tendrá que instalar en el entorno virtual usando `pip install SQLAlchemy`. Se van a definir dos modelos (Person y Address), cada uno de los cuales se asigna a una tabla, y luego vamos a completar la base de datos y realizar algunas consultas en ella. 

Comencemos con las declaraciones del modelo:

In [27]:
# pip install SQLAlchemy
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import (Column, Integer, String, ForeignKey, create_engine)
from sqlalchemy.orm import relationship

Al principio, importamos algunas funciones y tipos. Lo primero que tenemos que hacer entonces es crear un motor. Este motor le dice a `SQLAlchemy` sobre el tipo de base de datos que hemos elegido para nuestro ejemplo, y cómo conectarnos a ella:

In [31]:
engine = create_engine('sqlite:///:memory:')
Base = declarative_base()

class Person(Base):
    __tablename__ = 'person'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    age = Column(Integer)
    addresses = relationship(
        'Address',
        back_populates='person',
        order_by='Address.email',
        cascade='all, delete-orphan'
    )

    def __repr__(self):
        return f'{self.name}(id={self.id})'

class Address(Base):
    __tablename__ = 'address'
    id = Column(Integer, primary_key=True)
    email = Column(String)
    person_id = Column(ForeignKey('person.id'))
    person = relationship('Person', back_populates='addresses')

    def __str__(self):
        return self.email
    __repr__ = __str__

Base.metadata.create_all(engine)

C:\Users\rnico\AppData\Local\Temp\ipykernel_6396\620246408.py:2: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


A continuación, cada modelo hereda de la tabla `Base`, que en este ejemplo simplemente consta del valor predeterminado, devuelto por `declarative_base()`. Definimos `Person`, que se asigna a una tabla llamada `person` y expone los atributos id, name y age. También declaramos una relación con el modelo de `Address`, al indicar que al acceder al atributo `addresses` se obtendrán todas las entradas de la tabla de direcciones que estén relacionadas con la instancia de `Person` en particular con la que estamos tratando. La opción de `cascade` afecta al funcionamiento de la creación y la eliminación, pero es un concepto más avanzado, por lo que le sugerimos que lo ignore por ahora y tal vez investigue más más adelante.

Lo último que declaramos es el método `__rep __ ()`, que nos proporciona la representación oficial de una cadena de un objeto. Se supone que esta es una representación que se puede usar para reconstruir completamente el objeto, pero en este ejemplo, simplemente lo usamos para proporcionar algo en la salida. Python redirige `repr(obj)` a un llamado a `obj.__repr__ ()`.

También declaramos el modelo `Address`, que contendrá direcciones de correo electrónico y una referencia a la persona a la que pertenecen. Puede ver que los atributos person_id y person tienen que ver con la configuración de una relación entre las instancias `Address` y `Person`. Observe también cómo declaramos el método `__str__()` en `Address`, y luego le asignamos un alias, llamado `__repr__()`. Esto significa que llamar a `repr()` o `str()` en objetos `Address` resultará en última instancia en llamar al método `__str__()`. Esta es una técnica bastante común en Python, utilizada para evitar duplicar el mismo código, por lo que aprovechamos para mostrártela aquí. 

En la última línea, le decimos al motor que cree tablas en la base de datos de acuerdo con nuestros modelos.

Tener en cuenta que la función `create_engine()` admite un parámetro llamado `echo`, que se puede establecer en `True`, `False` o la cadena `"debug"`, para habilitar diferentes niveles de registro de todas las declaraciones y el `repr()` de sus parámetros.

Para una mayor comprensión de este código y base de datos, se recomienda que lea sobre los sistemas de administración de bases de datos (DBMS por sus siglás en inglés), SQL, álgebra relacional y SQLAlchemy. 

Continuando con el código, ahoa que tenemos nuestros modelos, vamos a usarlo para guardar algunos datos:

In [51]:
from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind=engine)
session = Session()

Primero creamos `Session`, que es el objeto que usamos para administrar la base de datos. A continuación, procedemos creando dos personas:

In [52]:
person_1 = Person(name='Juan Martinez', age=32)
person_2 = Person(name='Karla Quispe', age=40)

A continuación, añadimos direcciones de correo electrónico a ambos, utilizando dos técnicas diferentes. Uno los asigna a una lista y el otro simplemente los agrega:

In [53]:
person_1.addresses = [
    Address(email='juanmartinez32@example.com'),
    Address(email='jmartinez32@example.com'),
]

person_2.addresses.append(Address(email='karlaquispe40@example.com'))
person_2.addresses.append(Address(email='karla.quispe.40@example.com'))
person_2.addresses.append(Address(email='quispe.karla.40@example.com'))

Todavía no hemos tocado la base de datos. Es solo cuando usamos el objeto `session` que algo realmente sucede en él:

In [54]:
session.add(person_1)
session.add(person_2)
session.commit()

Agregar las dos instancias de `Person` es suficiente para agregar también sus direcciones (esto es gracias al efecto en cascada). Llamar a `commit()` es lo que realmente le dice a `SQLAlchemy` que confirme la transacción y guarde los datos en la base de datos. Una transacción es una operación que proporciona algo así como un espacio aislado, pero en un contexto de base de datos.

Mientras la transacción no se haya confirmado, podemos revertir cualquier modificación que hayamos realizado en la base de datos y, al hacerlo, volver al estado en el que nos encontrábamos antes de comenzar la transacción en sí. `SQLAlchemy` ofrece formas más complejas y granulares de lidiar con las transacciones, las cuales puedes estudiar en su documentación oficial, ya que es un tema bastante avanzado. Ahora consultamos a todas las personas cuyo nombre comienza con Jua usando `like()`, que se engancha al operador `LIKE` en SQL:

In [55]:
jua1 = session.query(Person).filter(
    Person.name.like('Jua%')
).first()

print(jua1, jua1.addresses)

Juan Martinez(id=1) [jmartinez32@example.com, juanmartinez32@example.com]


Tomamos el primer resultado de esa consulta (sabemos que solo tenemos a Juan Martinez de todos modos) y lo imprimimos. A continuación, obtenemos a Karla utilizando una coincidencia exacta con su nombre, sólo para mostrarte una forma diferente de filtrar:

In [56]:
karla = session.query(Person).filter(
    Person.name=='Karla Quispe'
).first()

print(karla, karla.addresses)

Karla Quispe(id=4) [karla.quispe.40@example.com, karlaquispe40@example.com, quispe.karla.40@example.com]


A continuación, capturamos el ID de Karla y eliminamos el objeto karla del marco global (esto no elimina la entrada de la base de datos):

In [57]:
karla_id = karla.id
del karla

La razón por la que hacemos esto es porque queremos mostrarle cómo obtener un objeto por su ID. Antes de hacer eso, escribimos la función `display_info()`, que usaremos para mostrar el contenido completo de la base de datos (obtenido a partir de las direcciones, para demostrar cómo obtener objetos usando un atributo de relación en `SQLAlchemy`):

In [58]:
def display_info():
    # Obtener todas las direcciones primero
    addresses = session.query(Address).all()
    
    # mostrar resultados
    for address in addresses:
        print(f'{address.person.name} <{address.email}>')

    # Mostrar cuántos objetos tenemos en total
    print('people: {}, addresses: {}'.format(
        session.query(Person).count(),
        session.query(Address).count())
    )

La función `display_info()` imprime todas las direcciones, junto con el nombre de la persona respectiva, y, al final, produce una información final sobre el número de objetos en la base de datos. Llamamos a la función, luego recuperamos y eliminamos karla. Por último, volvemos a mostrar la información, para verificar que realmente ha desaparecido de la base de datos:

In [59]:
display_info()
karla = session.get(Person, karla_id)
session.delete(karla)
session.commit()
display_info()

Juan Martinez <juanmartinez32@example.com>
Juan Martinez <jmartinez32@example.com>
Juan Martinez <juanmartinez32@example.com>
Juan Martinez <jmartinez32@example.com>
Juan Martinez <juanmartinez32@example.com>
Juan Martinez <jmartinez32@example.com>
Karla Quispe <karlaquispe40@example.com>
Karla Quispe <karla.quispe.40@example.com>
Karla Quispe <quispe.karla.40@example.com>
people: 4, addresses: 9
Juan Martinez <juanmartinez32@example.com>
Juan Martinez <jmartinez32@example.com>
Juan Martinez <juanmartinez32@example.com>
Juan Martinez <jmartinez32@example.com>
Juan Martinez <juanmartinez32@example.com>
Juan Martinez <jmartinez32@example.com>
people: 3, addresses: 6


Como se puede ver en los dos últimos bloques, al eliminar karla se ha eliminado un objeto `Person` y las tres direcciones asociadas a él. Una vez más, esto se debe al hecho de que la cascada tuvo lugar cuando eliminamos karla.

Con esto concluye nuestra breve introducción a la persistencia de datos. Es un dominio vasto y, a veces, complejo que te animamos a explorar, aprendiendo tanta teoría como sea posible. La falta de conocimiento o comprensión adecuada, cuando se trata de sistemas de bases de datos, puede ser realmente perjudicial.